In [1]:
# -*- coding: utf-8 -*-
"""
Created on Wed Mar 10 12:21:03 2019

@author: Vikas Singh
Let’s say you have a huge Twitter dataset. How will you build a word segmentation algorithm for the most trending HashTags.
For instance how will you decompose #IndianElections2019 to “Indian Elections 2019” or #100daysofcode to “100 days of code”.
"""
#Two Files count_1w.txt and count_2w.txt required
import re, string, random, glob, operator, heapq
from collections import defaultdict
from math import log10
from functools import reduce

def segment(text):
    text=text.lower()
    text=text.strip('#')
    "Return a list of words that is the best segmentation of text."
    if not text: return []
    candidates = ([first]+segment(rem) for first,rem in splits(text))
    return max(candidates, key=Pwords)

def splits(text, L=20):
    "Return a list of all possible (first, rem) pairs, len(first)<=L."
    return [(text[:i+1], text[i+1:]) 
            for i in range(min(len(text), L))]

def Pwords(words): 
    "The Naive Bayes probability of a sequence of words."
    return product(Pw(w) for w in words)
def product(nums):
    "Return the product of a sequence of numbers."
    return reduce(operator.mul, nums, 1)

class Pdist(dict):
    "A probability distribution estimated from counts in datafile."
    def __init__(self, data=[], N=None, missingfn=None):
        for key,count in data:
            self[key] = self.get(key, 0) + int(count)
        self.N = float(N or sum(self.itervalues()))
        self.missingfn = missingfn or (lambda k, N: 1./N)
    def __call__(self, key): 
        if key in self: return self[key]/self.N  
        else: return self.missingfn(key, self.N)

def datafile(name, sep='\t'):
    "Read key,value pairs from file."
    f = open(name, "r")
    for line in f:
        yield line.split(sep)

def avoid_long_words(key, N):
    "Estimate the probability of an unknown word."
    return 10./(N * 10**len(key))

N = 1024908267229 ## Number of tokens

Pw  = Pdist(datafile('count_1w.txt'), N, avoid_long_words)

#### segment2: second version, with bigram counts

def cPw(word, prev):
    "Conditional probability of word, given previous word."
    try:
        return P2w[prev + ' ' + word]/float(Pw[prev])
    except KeyError:
        return Pw(word)

P2w = Pdist(datafile('count_2w.txt'), N)

 
def segment2(text, prev='<S>'): 
    "Return (log P(words), words), where words is the best segmentation." 
    if not text: return 0.0, [] 
    candidates = [combine(log10(cPw(first, prev)), first, segment2(rem, first)) 
                  for first,rem in splits(text)] 
    return max(candidates) 

text='#IndianElections2019'
print(segment(text))
text='#100daysofcode'
print(segment(text))


['indian', 'elections', '2019']
['100', 'days', 'of', 'code']


In [2]:
text='#IndiavsAustralia'
print(segment(text))

['india', 'vs', 'australia']
